# Lab 9: Techniques for training Deep Neural Netwoks

```
- Machine Learning, Innopolis University (Fall semester 2022)
- Professor: Adil Khan
- Teaching Assistant: Gcinizwe Dlamini
```
<hr>


```
In this lab, you will practice techniques that are used to improve deep learning models perfomence in Pytorch.

Lab Plan
1. Data Augmentation examples
2. Batch normalization, Dropout, ...
3. Adaptive Learning rate and Optimizers
4. Using TensorBoard
5. Using Pretrained models (Transfer learning)

```
<hr>

# 1. CNN with PyTorch

## Load data and preprocess

To load our data set (CIFAR10) we can either download it manualy or use torchvision package. `torchvision` consists of popular datasets, model architectures, and common image transformations for computer vision.


For smooth training of the CNN we need to transform (normalize) the images. `transforms` package conatains common image transformations. Image transformations can be chained together using `transforms.Compose`. In our case we need to first convert image to tensor `transforms.ToTensor()` then normalize `transforms.Normalize`

The packages can be pip installed<br>
`!pip install torch` <br>
`!pip install torchvision`

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms

from torch.utils.tensorboard import SummaryWriter

batch_size = 128
test_batch_size = 100

# Transformations
# TODO : add data augmentation of your choice
transform_train = transforms.Compose(
    [transforms.ToTensor(),
     None,
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

transform_test = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Create Train Dataloader
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

#Create Test Dataloader
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=test_batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


## Data visualization 
Create a simple method to have a look at the data (image)

In [3]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images[:4]))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "D:\Innopolis\_my\env\lib\site-packages\torch\utils\data\_utils\worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "D:\Innopolis\_my\env\lib\site-packages\torch\utils\data\_utils\fetch.py", line 49, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "D:\Innopolis\_my\env\lib\site-packages\torch\utils\data\_utils\fetch.py", line 49, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "D:\Innopolis\_my\env\lib\site-packages\torchvision\datasets\cifar.py", line 118, in __getitem__
    img = self.transform(img)
  File "D:\Innopolis\_my\env\lib\site-packages\torchvision\transforms\transforms.py", line 94, in __call__
    img = t(img)
TypeError: 'NoneType' object is not callable


## Lets Switch to the GPU (if available) - why?


Unlike TensorFlow, PyTorch doesn’t have a dedicated library for GPU users,so its neccessary to some manual setup <br>
<b>NB: </b>If working on Colab, make sure that GPU runtime is enabled <br>
<b>NB: </b>It’s not possible to transfer Data Loaders directly to GPU <br>
<b>NB: </b> To get total number of GPU devices available `torch.cuda.device_count()`

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assume that we are on a CUDA machine, then this should print a CUDA device:

print(device)

cpu


## Create a Simple Convolutional neural network (CNN)

The simple CNN is made up of 2D Convolutional layers, Pooling layers and fully connected layers. <br> 
<b>NB : </b> If we succesfully connected to GPU, we need to send our CNN model to GPU for faster training

In [ ]:
#TODO: Create the BaseModel by defining the missing (None) convolutional layers, Max pool and fully connected layers
#TODO: Finish implementation of foward method in class BaseModel
import torch.nn as nn
import torch.nn.functional as F

class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1) 
        
        # TODO 
        self.conv2 = nn.Conv2d(None, 64, 3, padding=1) 
        self.pool1 = nn.MaxPool2d(2, 2) 
        
        # TODO 
        self.conv3 = nn.Conv2d(None, 128, 3, padding=1) 
        self.pool2 = nn.MaxPool2d(2,2)
        
        # TODO 
        self.fc1 = nn.Linear(None, 256)
        self.fc2 = nn.Linear(256, None)
        self.output = nn.Linear(84, 10)

    def forward(self, x):
        # TODO: Implement the foward pass method. NOTE: Apply Relu activation function to all layers 
        # order : conv1 -> activation -> conv2 -> activation -> pool1 -> conv3 -> activation -> pool2 -> fc1 -> activation -> fc2 -> activation -> output
        # x = F.relu
        pass

model = BaseModel().to(device)

In [5]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [6]:
print("Number of params in the CNN model:", count_parameters(model))

Number of params in the CNN model: 2213094


## 3. Training and testing

In [7]:
import torch.optim as optim
def train_evaluate(net, optimizer):
    criterion = nn.CrossEntropyLoss()
    
    for epoch in range(10):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(trainloader, 0): #TODO: Iterate through the batches and train the cnn
            # TODO : get the inputs and place them to the training devide (i.e GPU or CPU) 
            inputs, labels = None
            inputs, labels = None
            
            # TODO : Train procedure -> zero the gradients, forward pass + backward pass + optimize

            # forward + backward + optimize
            outputs = None
            loss = None
            
            running_loss += loss.item()
        
        # TODO: print statistics and log loss to Tensorboard
        running_loss /= len(trainloader)
        print(f'[Epoch {epoch+1}] Loss {running_loss:.4f}')
                
    print('Finished Training')
    
    #Print Model overall Accuracy 
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))
    
    #Print Model classwise Accuracy 
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(4):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1


    for i in range(10):
        print('Accuracy of %5s : %2d %%' % (
            classes[i], 100 * class_correct[i] / class_total[i]))

## Simple CNN training and Evaluation

In [8]:
# TODO : define an optimizer SGD with learning rate 0.001 and momentum 0.9
optimizer = None
train_evaluate(model, optimizer)

[Epoch 1] Loss 2.3017
[Epoch 2] Loss 2.2940
[Epoch 3] Loss 2.2453
[Epoch 4] Loss 2.0382
[Epoch 5] Loss 1.8688
[Epoch 6] Loss 1.7389
[Epoch 7] Loss 1.6463
[Epoch 8] Loss 1.5832
[Epoch 9] Loss 1.5364
[Epoch 10] Loss 1.4968
Finished Training
Accuracy of the network on the 10000 test images: 48 %
Accuracy of plane : 61 %
Accuracy of   car : 75 %
Accuracy of  bird : 23 %
Accuracy of   cat : 30 %
Accuracy of  deer : 28 %
Accuracy of   dog : 40 %
Accuracy of  frog : 64 %
Accuracy of horse : 62 %
Accuracy of  ship : 47 %
Accuracy of truck : 31 %


## Improved CNN Model

### Task 
1. Add Dropout for both convolution and fully connected layers
2. Add Batch Normalization for convolutions
3. Add Batch Normalization for fully connected network

In [9]:
#TODO: Create the ImprovedModel by defining the missing (None) batch Normalization layers and Dropout
class ImprovedModel(nn.Module):
    def __init__(self):
        super(ImprovedModel, self).__init__()
        self.network=nn.Sequential(
            nn.Conv2d(3,32,3,padding=1),
            None , # Batch Normalization
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            None,  # Dropout 25%
            
            nn.Conv2d(32,64,3,padding=1),
            None, # Batch Normalization
            nn.ReLU(),
            None,  # Dropout 25%
            
            nn.Conv2d(64,128,3,padding=1),
            None, # Batch Normalization
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            None,  # Dropout 25%
            
            nn.Conv2d(128,128,3,padding=1),
            None , # Batch Normalization
            nn.ReLU(),
            nn.Dropout(0.25),
            
            nn.Conv2d(128,128,3,padding=1),
            None, # Batch Normalization
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            None, # Dropout 25%
            
            nn.Flatten(),
            nn.Linear(None),
            nn.ReLU(),
            nn.Linear(512,256),
            nn.ReLU(),
            nn.Linear(256,10))

    def forward(self, x):
        return self.network(x)

model_improved = ImprovedModel().to(device)

In [10]:
print("Number of params in the CNN model:", count_parameters(model))

Number of params in the CNN model: 2213094


## Improved CNN training and Evaluation

In [11]:
optimizer = torch.optim.Adam(model_improved.parameters(),lr=0.001)
train_evaluate(model_improved, optimizer)

[Epoch 1] Loss 1.5229
[Epoch 2] Loss 1.2268
[Epoch 3] Loss 1.0909
[Epoch 4] Loss 1.0057
[Epoch 5] Loss 0.9471
[Epoch 6] Loss 0.8981
[Epoch 7] Loss 0.8546
[Epoch 8] Loss 0.8245
[Epoch 9] Loss 0.7930
[Epoch 10] Loss 0.7744
Finished Training
Accuracy of the network on the 10000 test images: 72 %
Accuracy of plane : 84 %
Accuracy of   car : 87 %
Accuracy of  bird : 52 %
Accuracy of   cat : 72 %
Accuracy of  deer : 61 %
Accuracy of   dog : 68 %
Accuracy of  frog : 88 %
Accuracy of horse : 84 %
Accuracy of  ship : 64 %
Accuracy of truck : 79 %


## CNN from popular model architectures (Transfer learning)

Same procedure as simple CNN but the achitecture of the neural net is much more complicated and the weights are pretrained. <br> 


### Fine Tuning
Fine tuning or transfer learning is the task of training a model on a big dataset and then adjusting the parameters of the model for a smaller task with less data.

It is very common in Computer Vision and Natural Language Processing with the immergence of BERT and UMLfit. <br>
![](https://miro.medium.com/max/1276/1*ZkPBqU8vx2vAgcLpz9pi5g.jpeg)

Say we want to use resnet (trained on imagenet with 1000 classes and has input shape of 256x256)  and fine-tune it for CIFAR10 (has 10 categories with input shape of 32×32)

What is the biggest changes that we need to make?



## How to Import Popular CNN models? 

`torchvision,models` contains of popular model architectures and can be loaded together with their trained weights


In [12]:
import torchvision.models as models

# resnet18 = models.resnet18(pretrained=True,progress=True)
# alexnet = models.alexnet(pretrained=True,progress=True)
# squeezenet = models.squeezenet1_0(pretrained=True,progress=True)
# vgg16 = models.vgg16(pretrained=True,progress=True)
# densenet = models.densenet161(pretrained=True,progress=True)
# inception = models.inception_v3(pretrained=True,progress=True)
# googlenet = models.googlenet(pretrained=True,progress=True)
# shufflenet = models.shufflenet_v2_x1_0(pretrained=True,progress=True)
# mobilenet = models.mobilenet_v2(pretrained=True,progress=True)
# resnext50_32x4d = models.resnext50_32x4d(pretrained=True,progress=True)
# wide_resnet50_2 = models.wide_resnet50_2(pretrained=True,progress=True)
# mnasnet = models.mnasnet1_0(pretrained=True,progress=True)

## Start by loading and transforming the data 

In [13]:
import torchvision.models as models

transform = transforms.Compose(
    [transforms.Resize((256,256)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## Define CNN model 

<b>NB : </b>To make the model architecture fit our classification task, we will change the last fully connected layer and train only it.

In [14]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.net = models.resnet18(weights='IMAGENET1K_V1', progress=True)
        self.net.trainable = False # Freeze the layers of the pretrained model
        self.net.fc = nn.Linear(512, 10)

    def forward(self, x):
        return self.net(x)


net = Net()
net = net.to(device)

## Self-practice Task


1. Add Tensorboard to monitor the train and validation loss, recall, precision and accuracy every after training epoch
2. Create another CNN using pretrained model and compare it with already implemented resnet and simple CNN (plot their losses on same tensorboard)